# Creating websites using flask

## flask

### Intro

* a program written using flask functions as a web-server;
* each function is responsible for returning the html-code of a page;
* functions can use html code from files;
* those files can contain pages or templates for pages

Example website written using flask:

(to run the program type `python path/filename` in the command line/terminal)

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return '<html><body><p>Hello, world!</p></body></html>'

if __name__ == '__main__':
    app.run(debug=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2018 16:23:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2018 16:23:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/May/2018 16:23:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/May/2018 16:23:17] "GET / HTTP/1.1" 200 -


Every webpage is generated by a function. The decorator `@app.route(...)` before the function shows the address of the webpage this function is responsible for:

In [ ]:
@app.route('/')
def index():
    return 'Main page'

@app.route('/hi')
def hi():
    return 'Hi!'

One function can be responsible for several webpages:

* first, there could be several decorators in a row;
* second, the decorators can contain variables.

An example:

In [ ]:
@app.route('/user/<user>')
def user_index(user):
    return 'This is the page of' + user

The variables in the address could be of different types: `int` — an integer, `float` — a float, `path` — a string. An example to show how you could specify the variable type:

In [ ]:
import datetime

@app.route('/time/<int:shift>')
def time_page(shift):
    h = datetime.datetime.today().hour
    h += shift
    return 'Time in your country:' + str(h)

To learn the address of a webpage use the `url_for` function:

In [ ]:
from flask import url_for

@app.route('/functions/<fname>')
def f_address(fname):
    return 'The address is ' + url_for(fname)

### Once more about HTML forms

* HTML-forms are used to interact with the server
* the form should have a `Submit` button
* forms can have text fields, checkbox fields etc.
* the data entered into the form is sent to the server with the GET or the POST methods
* if the GET method is used, the data is appended to the URL after `?`

In [ ]:
<form>
    Имя: <input type="text" name="name"><br>
    Возраст: <input type="text" name="age"><br>
    Пароль: <input type="password" name="pwd"><br>
    <input type="checkbox" name="student"> студент
    <input type="submit" value="Отправить">
</form>

When you click on the `submit` button (assuming the GET method is used), a request is sent to the server, the URL of the request contains all the values of the parameters after a `?`:

    www.example.com:5000/some_page.html?name=Petya&age=&student=on

All of the symbols in the URL, except for the Latin symbols, will be encoded with percent encoding:

    name=%D0%A2%D0%B8%D0%BC%D0%BE%D1%84%D0%B5%D0%B9

To use the data from the form in a flask application, you need to use the `request` object:
* `request.method` — the method of the request
* `request.args` — a dictionary that contains the values of the arguments

In [ ]:
from flask import request
@app.route('/login')
def login():
    if request.args['password'] == '123':
        return 'Name: ' + request.args['login']


### Templates

Dynamic webpages use templates. All the `templates` should be put into the templates folder.

A template is created using the html-code.

HTML fragments inside `{% ... %}` or `{{ ... }}` contain variables or commands that use variables. 

To work with templates we will need the `render_template()` function:

In [ ]:
from flask import render_template
@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('hello.html', name=name)

* The `render_template` function substitutes anything inside the brackets with the corresponding html code. 
* The variables are submitted to the template with `render_template`.

There are two types of insertions: `{{ ... }}` and `{% ... %}`.

__Fragments `{{ ... }}`__

* inside the brackets we can have a variable: `{{ name }}`
* you cannot apply random functions to those variables: `{{ f(name) }}`
* there are built-in operations that you can apply to the variables
* those operations are separated from the variable with |, e.g., `{{ name|length }}`
* some examples of the operations:
    * `length`
    * `lower`, `upper`
    * `random` — to select a random element from an array
    * `urlencode` — encode a string with percent encoding

__Fragments `{% ... %}`__

Commands similar to the `if` and `for` commands in python are put inside `{% ... %}`:

`{% for i in arr %} ... {% endfor %}`

`{% if ... %} ... {% endif %}`

`{% elif ... %}`

`{% else %}`


In [ ]:
{% if username|length > 20 %}
    <p>The name is too long!</p>
{% else %}
    <p>{{ username }}</p>
{% endif %}

## Templates with dictionaries

Suppose that we have a dictionary that contains our friends' names and e-mails. We want to display their names and e-mails on our webpage.

In [ ]:
from flask import Flask
from flask import render_template

app = Flask(__name__)


@app.route('/')
def index():
    emailbook = {'Петя': 'petya@example.com',
                 'Вася': 'vasya@example.com',
                 'Катя': 'katya@example.com'}
    return render_template('index.html', emails=emailbook)

if __name__ == '__main__':
    app.run()

Then, in the templates folder, we need to create a file index.html, where we would go through the elements of the dictionary. To do so, use the function `items()`. Here is our index.html:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Почтовые адреса</title>
</head>
<body>
<h1>Адреса моих друзей</h1>
<ul>
    {% for name, email in emails.items() %}
        <li>{{ name }} - {{ email }}</li>
    {% endfor %}
</ul>
</body>
</html>

## Redirection

To redirect the user to a different page, use the function `redirect`. For example, below you can see an application with the page /time. We could do the following:

- if the user visits the page from 10am to 6pm, they are redirected to the main page;
- else, they are redirected to the page /hi.

In [ ]:
import datetime

from flask import Flask
from flask import url_for, render_template, request, redirect

app = Flask(__name__)


@app.route('/')
def index():
    return '<html><body><h1>Привет, мир!</h1></body></html>'


@app.route('/hi')
@app.route('/hi/<user>')
def hi(user=None):
    if user is None:
        user = 'friend'
    return '<html><body><p>Привет, ' + user + '!</p></body></html>'


@app.route('/time')
def time_redirect():
    h = datetime.datetime.today().hour
    if 10 < h < 18:
        return redirect(url_for('index'))
    return redirect(url_for('hi'))


if __name__ == '__main__':
    app.run(debug=True)

Use `debug=True` while you are working on your website, it allows you to see the errors. When you put your application on the Internet, don't forget to change it to `debug=False`.

## How do we display the contents of a text file on our webpage?

In [ ]:
from flask import Flask
from flask import url_for, render_template, request, redirect

app = Flask(__name__)

@app.route('/poem')
def poem():
    with open("poem.txt", "r", encoding='utf-8') as f:
        content = f.read().split('\n')
    return render_template("poem.html", content=content)

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>Зимнее утро</title>
</head>
<body>
    <p>
    {%for i in content%}
        {{i}}
        <br>
    {%endfor%}
    </p>
</body>
</html>